## Adquisición con osciloscopio Tektronix TBS1000C

En este tutorial vamos a ver:

1- Cómo utilizar la [librería pyvisa](https://pyvisa.readthedocs.io/en/latest/) de python para comunicarnos con el equipo

2- Cómo adquirir una curva mostrada por el osciloscopio

3- Cómo realizar mediciones sobre la señal observada

4- Cómo obtener las incertezas en la medición


[ver manual de programación del TBS1000C](https://drive.google.com/file/d/12OwuZ7jb8mmm6hDEhMLc-YBIp1Tau8EP/view?usp=sharing)


[ver manual de usuario del TBS1000C](https://drive.google.com/file/d/1qHF2r6Nui-zS7caoaPUTwJCwMAxECCAU/view?usp=sharing)

### 1- Cómo utilizar pyvisa para comunicarnos con el equipo

Empezamos importando las librerías que vamos a utilizar:

In [ ]:
import pyvisa                               # para comunicarnos con los equipos
import numpy as np                          # para operar con listas de datos
import pandas as pd                         # para manejar archivos de datos: Guardar, abrir, etc
import matplotlib.pyplot as plt             # para plotear 
import time                                 # para realizar pausas entre mediciones

En primer lugar, necesitamos definir un objeto que nos permita comunicarnos e interactuar con los instrumentos que tenemos conectados vía puerto USB, GPIB, Serial (RS232), Ethernet, etc. Este objeto se conoce como Resource Manager y lo invocamos desde la librería pyvisa:

In [ ]:
rm = pyvisa.ResourceManager()            # Defino objeto resource manager de pyvisa

Este se utiliza para: (a) detectar dispositivos conectados y listar todos los recursos disponibles y (b) crear interfases para comunicarnos con dispositivos.

Una vez definido el objeto **rm**, podemos listar los instrumentos conectados a nuestra PC: 

In [ ]:
instrumentos_conectados = rm.list_resources()           # lista de instrumentos conectados

print (instrumentos_conectados)

De la lista obtenida en la celda anterior, vemos que cada instrumento es identificado con un *string* de la siguiente manera: `puerto::marca::modelo::NroSerie`. Para identificar cual de todos es el osciloscopio, buscamos el *modelo* en la etiqueta ubicada en la parte trasera del equipo. Una vez identificado, copiamos el nombre y lo pegamos en la variable *instrumento*, luego procedemos a establecer la comunicación con el equipo:

In [ ]:
instrumento = 'puerto::marca::modelo::NroSerie'         # colocar acá el instrumento asociado al osciloscopio 

osci = rm.open_resource(instrumento)                    # Establezco comunicación con el osciloscopio

osci.query('*IDN?')                                     # le pido que se identifique para checkear la conexión

El objeto *osci* será nuestra interfase, éste nos permite interactuar con el osciloscopio utilizando rutinas estándar. Generalmente, realizamos dos tipos de operaciones:

- Enviar un comando al instrumento para configurar parámetros o iniciar procesos sin esperar ninguna respuesta. Esto se realiza mediante rutinas de tipo **write**.
  
- Enviar un comando al instrumento y esperar una respuesta, como por ejemplo, obtener los datos que estoy midiendo o consultar el estado del instrumento. Esto se realiza mediante rutinas de tipo **query**.

El lenguaje que utilizamos para comunicarnos con el equipo está definido en el estándar [**SCPI**](https://en.wikipedia.org/wiki/Standard_Commands_for_Programmable_Instruments) (Standard Commands for Programmable Instruments), el cual es ampliamente utilizado para la comunicación remota con equipos de instrumentación como osciloscopios, multímetros y generadores de funciones.

Una vez que comprobamos la conexión con el equipo, lo primero que haremos es configurar como queremos que éste formatee los datos de salida. Empezamos ejecutando,

In [ ]:
osci.write('DAT:ENC RPB')

Con ese comando le indicamos al equipo cómo debe codificar los datos de salida. En este caso, utilizaremos datos **Raw Binary** (RPB), lo que significa que queremos los datos en formato binario crudo, sin ningún tipo de procesamiento o compresión.

A continuación, seteamos el ancho de banda en *1 byte*. Es decir, establecemos que el osciloscopio envíe datos como valores de 8 bits por muestra:

In [ ]:
osci.write('DAT:WID 1')         # data band width 1 byte

Pedimos también que los datos sean transmitidos limpiamente, sin encabezados:

In [ ]:
osci.write('HEADER OFF')

### 2. Adquisición de una curva mostrada por el osciloscopio

Supongamos que estás utilizando el osciloscopio para medir la caída de tensión en un componente de tu circuito y estás visualizando la curva de tensión en uno de los canales del equipo. Te interesa adquirir esa curva para realizar algún tipo de análisis de datos en la PC. A continuación, te explico cómo hacerlo.

Primero, encendemos el canal del equipo con el que queremos adquirir, en este caso, el canal 1 (CH1):

In [ ]:
osci.write('SEL:CH1 ON')        # select:channel1 ON

configuramos el equipo en AUTOSET. Esto le indica al osciloscopio que ajuste su configuración para optimizar la visualización de la señal que está midiendo, en particular nos interesa que realice un ajuste automático de la escala. Para esto, escribimos:

In [ ]:
osci.write('AUTOS EXEC')   # Equivalente a apretar el botón AUTOSET del equipo, 

time.sleep(5)                   

La rutina *time.sleep()* genera una pausa de 5 segundos, tiempo de espera necesario para que el equipo optimice la visualización de la señal antes de poder recibir otra orden.


Si estamos por tomar los datos y la pantalla se mueve mucho, por ejemplo por una señal ruidosa, es conveniente congelar la pantalla antes de medir. Eso se hace con el comando:

In [ ]:
osci.write('ACQ:STATE OFF')     

Configuramos el osciloscopio para que la fuente de datos sea el canal 1 (CH1). Esto significa que, al realizar operaciones de adquisición o cálculos posteriormente, el osciloscopio usará la señal medida en el canal 1:

In [ ]:
osci.write('DAT:SOU CH1')       # data:source channel1

Una vez fijada la fuente de datos y congelado la pantalla, solicitamos al osciloscopio algunos parámetros relacionados con la escala de la señal que estamos adquiriendo. 

In [ ]:
xze, xin, yze, ymu, yoff = osci.query_ascii_values('WFMOUTPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;',separator=';') # waveform preamble information

time.sleep(2)           # breve pausa

Siempre antes de hacer un *query* es recomendable hacer una pausa para darle tiempo al equipo a que termine de configurar las ordenes anteriores. En el contexto del *Jupyter Notebook* donde uno va ejecutando de a uno las celdas del programa, esto pude que no sea necesario, ya que esa misma dinámica le da las pausas necesarias. No obstante cuando uno decide correr todas las celdas juntas, la pausa con el *time.sleep()* cobra relevancia.


En el comando múltiple *osci.query_ascii_values()* estamos solicitando al equipo: 

- 'WFMOUTPRE:XZE?': Pide el desplazamiento horizontal inicial (XZE), que es el tiempo correspondiente al primer punto de la señal.
- 'WFMOUTPRE:XIN?': Pide el intervalo de muestreo (XIN), que es el tiempo entre muestras consecutivas.
- 'WFMOUTPRE:YZE?': Pide el desplazamiento vertical cero (YZE), que es el nivel de voltaje que corresponde a un valor digital de cero.
- 'WFMOUTPRE:YMU?': Pide la escala vertical (YMU), que convierte unidades digitales a unidades físicas (típicamente voltios).
- 'WFMOUTPRE:YOFF?': Pide el desplazamiento vertical (YOFF), que corrige la señal para que el cero físico corresponda al cero digital.

Más adelante utilizaremos estos parámetros para escribir la curva en términos de las magnitudes físicas correspondientes.

Ahora si, finalmente, adquirimos la curva que observamos en el osciloscopio:

In [ ]:
time.sleep(2)           

dataCH1 = osci.query_binary_values('CURV?', datatype='B', container=np.array) 

Los argumentos de la función *osci.query_binary_values()* indican:

- 'CURV?': Es la consulta que se realiza al dispositivo. En este caso, 'CURV?' generalmente se usa para solicitar datos de forma de onda (curva) del osciloscopio. Esto indica que el dispositivo debe enviar datos binarios de la forma de onda.

- datatype='B': Especifica el tipo de datos que se esperan recibir del dispositivo. 'B' indica que los datos se recibirán como bytes (es decir, valores enteros de 8 bits sin signo).

- container=np.array: Especifica el contenedor en el que se deben almacenar los datos recibidos. *np.array* indica que los datos se almacenarán en un array de NumPy.

Ya con la curva adquirida, utilizamos los parámetros para re-escribirla en términos de las magnitudes físicas correspondientes: 

In [ ]:
t = xze + np.arange(len(dataCH1)) * xin             # Conversion a tiempo en segundos

VCH1 = (dataCH1 - yoff) * ymu + yze                 # Conversion a voltaje en Volts

Hacemos un gráfico para visualizar los datos:

In [ ]:
fig, ax  =  plt.subplots()

ax.plot(t, VCH1, label="CH1")

ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Voltaje [V]')
plt.show()

Queremos guardar los datos medidos, una forma sencilla es hacerlo a través de la librería *pandas*. Primero definimos un objeto *DataFrame* al cual le pasamos como argumentos los datos en un diccionario:

In [ ]:
data = {"tiempo":t, "voltaje":VCH1}

df = pd.DataFrame( data=data ) 

df.head()                                              # muestro las primeras filas de datos 

El objeto resultante, *df*, es un tipo de estructura de datos en Pandas que funciona como una tabla, similar a una hoja de cálculo o una base de datos, donde a partir de varias rutinas de la librería, podemos almacenar y manipular datos organizados en filas y columnas.

Hecho esto, definimos una carpeta de destino para los datos, un nombre, y guardamos el conjunto de datos en un archivo tipo *Comma Separated Values* (*.csv*):

In [ ]:
carpeta = "path/to/you/folder/"                       # indicar dirección de la carpeta donde se guardarán los datos

df.to_csv( carpeta+"voltaje_CH1.csv", index=False )

Adquirida la curva y guardada, vuelvo a activar la adquisición para seguir trabajando en otras mediciones:

In [ ]:
osci.write('ACQ:STATE ON')

### 3. Mediciones matemáticas sobre la señal

Estamos midiendo la caída de tensión sobre un componente de nuestro circuito. Para esto estamos utilizando el canal 1 del osciloscopio (CH1). Visualizamos la curva en el osciloscopio y queremos obtener de ésta la amplitud y la fase. Para realizar estas mediciones, vamos a utilizar el comando **MEASUREMENT:IMMED**.

#### 3.1. Medición de amplitud de la señal

Antes de medir ajusto la escala del osciloscopio y le doy 5 segundos de pausa antes de ejecutar un nuevo comando

In [ ]:
osci.write('AUTOS EXEC')

time.sleep(5)               # esta pausa da tiempo para que ocurra la auto configuración

En primer lugar configuramos al canal 1 como la fuente para una medición inmediata en el osciloscopio:

In [ ]:
osci.write('MEASUrement:IMMed:SOURce CH1')              # las letras de los comandos en minúsculas no es necesario ponerlas

Para medir la amplitud, configuramos al osciloscopio para que la medición sea del tipo *Root Mean Square* (RMS Ciclo), es decir pedimos que cuando midamos nos entregue el voltaje eficaz de la señal:

In [ ]:
osci.write('MEASUrement:IMMed:TYPe CRMS')

Mido la amplitud en el canal 1:

In [ ]:
ACH1= float( osci.query('MEASUrement:IMMed:VALue?') ) 

time.sleep(2)

Note que *osci.query('MEASUrement:IMMed:VALue?')* devuelve un string con el valor medido, como por ejemplo "0.1234" o "1.56E-03". Para convertirlo a un número flotante y poder hacer cálculos con él, es necesario usar la rutina *float()*.

Muchas veces puede ser útil medir la amplitud pico a pico de la señal, para hacerlo, por ejemplo en el canal 1, ejecutamos:

In [ ]:
osci.write('MEASUrement:IMMed:SOURce CH1')

osci.write('MEASUrement:IMMed:TYPe PK2pk')

Apk_CH1= float( osci.query('MEASUrement:IMMed:VALue?') ) 

time.sleep(2)

#### 3.2. Medición de la fase de la señal

Al igual que antes utilizamos el comando MEASU:IMM, el procedimiento es similar al caso de la amplitud: 

Definimos una medición inmediata con fuente en el canal 1, configuramos el tipo de medición en *tipo fase*, y ejecutamos un *query* para adquirir el valor:

In [ ]:
osci.write('MEASU:IMM:SOUR CH1')

osci.write('MEASU:IMM:TYP PHAse')

time.sleep(2)

fase = osci.query_ascii_values('MEASUrement:IMMed:VALue?')[0]      # en grados

### 4. Acerca de los errores en la medición

#### 4.1 Incerteza en la medición de voltaje


Para obtener la incertidumbre en la medición de la amplitud, para eso utilizamos la expresión:


$\Delta A = \Delta A_{\text{rel}} + \Delta A_{\text{res}}$

Donde:  
- $ \Delta A_{\text{rel}}$: Error relativo asociado al sistema de medición.  
- $ \Delta A_{\text{res}}$: Error debido a la resolución del sistema (cuantización).

Cálculo de cada término:

1. **Error relativo ($ \Delta A_{\text{rel}}$)**  
    $\Delta A_{\text{rel}} = \alpha ACH1$
    
    Donde:  
    - $ACH1$ es la amplitud medida.
    - $\alpha$ es el error relativo dado por el fabricante del dispositivo (en este osciloscopio, $\alpha = 0.03$ o $3\%$).

2. **Error de resolución ($\Delta A_{\text{res}}$)**  
   $\Delta A_{\text{res}} = \frac{\text{escalav1} \times (R/N)}{2}$

   Donde:  
   - $\text{escalav1}$ es la escala de medición (por ejemplo 0.1 V/div).  
   - $R$ es el rango total del sistema, es la diferencia entre el máximo y el mínimo voltaje que el osciloscopio puede medir.
    En este modelo R=10 V (mide de -5 V a +5 V).  
   - $N$ es el número de niveles de cuantización, como estoy midiendo con 8 bits, N = 255.
   - El factor 2 indica estamos considerando que el error de resolución podría ser de hasta la mitad del intervalo de cuantización.

Note que, necesitamos la escala vertical (o sensibilidad) del canal 1 para realizar el cálculo. Para obtenerla ejecutamos:

In [ ]:
escalav1 = float(osci.query('CH1:SCAle?'))

El error de medición entonces se calcula como:

In [ ]:
errACH1 = 0.03*ACH1 + (escalav1*(10/255))/2.0

#### 4.2 Incerteza en la medición de tiempo


**Presición de la base de tiempo**. Mide la estabilidad y precisión del oscilador de cristal interno del osciloscopio que define el paso del tiempo. Para este equipo es de $25ppm$, significa que por cada 1 millón de segundos que el osciloscopio intenta medir, puede tener un error de hasta 25 segundos. Se aplica al intervalo de tiempo total, $t$, que estás midiendo,

$$
\Delta t_{bt} = t \times 25\times10^{-6} 
$$


**Incertidumbre por Discretización Temporal**. El osciloscopio digital no mide la señal de forma continua; la captura a intervalos fijos de tiempo, determinados por la Tasa de Muestreo (Sample Rate, $f_s$). Cada punto muestreado se "bloquea" en un instante de tiempo específico. El intervalo de tiempo entre dos puntos de muestreo adyacentes se llama Intervalo de Muestreo, $t_{step} = 1/f_s$. Se puede probar que el error asociado a esta discretización en este equipo viene dado por,

$$
\Delta t_{dis} = \frac{t_{step}}{\sqrt{12}} 
$$​

La frecuencia de muestreo puede depender de la escala, por lo tanto cada vez que medimos debemos solicitarla vía comandos, 

In [ ]:
fs = float( osci.query('HORizontal:SAMPLERate?') )

err_t_dis = (1/fs)/np.sqrt(12)

Los dos errores anteriores son muy chicos y en general no aportan mucho. En lo que sigue describimos el ruido de canal, que aparece cuando uno quiere medir tiempos asociados a cruces de umbral.

**Incertidumbre por ruido**. La incerteza de timing inducida por el canal (Jitter) es la imprecisión con la que el osciloscopio detecta el instante exacto de tiempo (t) para un evento (como un cruce por cero o un flanco), y tiene que ver con los errores en el eje vertical. Viene dado por,

$$
\Delta t_{ruido} = \big| \frac{ \Delta V_{ruido.rms} }{ \frac{dV}{dt} } \big|
$$

Donde el numerador se puede obtener midiendo la desviación estandar y el denominador se puede aproximar utilizando la amplitud de la señal, por ejemplo para medir este ruido en el canal 1,

In [ ]:
osci.write('MEASUrement:IMMediate:SOUrce1 CH1')
osci.write('MEASUrement:IMMediate:TYPe RMS')
DV_ruido_rms = osci.query_ascii_values('MEASUrement:IMMed:SDEv?')[0]
time.sleep(2)


osci.write('MEASUrement:IMMediate:SOUrce1 CH1')
osci.write('MEASUrement:IMMediate:TYPe SLEWrate')
dV_dt = osci.query_ascii_values('MEASUrement:IMMediate:VALue?')[0]
time.sleep(2)

err_t_ruido = abs( DV_ruido_rms/ dV_dt )

En señales periodicas, la deriva puede ser aproximada por $ \frac{dV}{dt} \approx 2\pi f A_{pk} $, donde $A_{pk}$ es la amplitud pico a pico.

#### 4.3 Incerteza en la medición de fase

Para obtener la incerteza de la fase conviene medir el promedio y la desviación estandar,

In [ ]:
osci.write('MEASUrement:IMMediate:TYPe PHASE')
osci.write('MEASUrement:IMMediate:SOUrce1 CH1')

# acá necesito esperar un tiempo para que el equipo acumule estadística
time.sleep(5)

fase_mean = osci.query_ascii_values('MEASUrement:IMMediate:MEAN?')[0]
std_fase  = osci.query_ascii_values('MEASUrement:IMMediate:SDEv?')[0]

